In [2]:
import os, ast
from graphviz import Digraph

project_root = "src"
graph = Digraph(comment="Python File Dependencies")

# map file path → module name
files = {
    os.path.relpath(os.path.join(dp, f), project_root).replace(os.sep, ".").rstrip(".py"): os.path.join(dp, f)
    for dp, _, filenames in os.walk(project_root)
    for f in filenames if f.endswith(".py")
}

def get_imports(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        tree = ast.parse(f.read(), filename=file_path)
    imports = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for n in node.names:
                imports.append(n.name)
        elif isinstance(node, ast.ImportFrom):
            if node.module:
                imports.append(node.module)
    return imports

for module, path in files.items():
    imports = get_imports(path)
    for imp in imports:
        for target in files:
            if imp == target or imp.startswith(target + "."):
                graph.edge(module, target)

graph.render("file_dependencies", format="svg", view=True)


'file_dependencies.svg'